### Задание 1

In [2]:
import torch
import pandas as pd
from sklearn.model_selection import train_test_split
import time
import random
import os
import re
import sys
from google.colab import drive
import copy

dev = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
dev

device(type='cuda')

In [3]:
def Сaesar(string, num):
    output = ''
    for c in string:
        if c.isalpha():
            new_num = ord(c) + num
            if new_num > ord('z'):
                new_num -= 26
            output += chr(new_num)
        else:
            output += c
    return output

In [4]:
Сaesar('test', 2)

'vguv'

In [6]:
MAX_LEN = 50
ABC = ['none'] + [w for w in set('abcdefghijklmnopqrstuvwxyz ')]
CHAR_TO_INDEX = {w: i for i, w in enumerate(ABC)}
INDEX_TO_CHAR = {i: w for i, w in enumerate(ABC)}

df = pd.read_csv('/content/data.csv').iloc[:,[-2]]
df = df.dropna(subset=['normalized_text'])

df['text_in'] = [' '.join(re.findall('[\w]+', i)) for i in df['normalized_text']]
df['text_out'] = df['text_in'].apply(lambda v: Сaesar(v, 2))

df = df.iloc[:, 1:]
df.head()

,text_in,text_out
0,maggie look whats that,ociikg nqqm yjcvu vjcv
1,lee mur lee mur,ngg owt ngg owt
2,zee boo zee boo,bgg dqq bgg dqq
3,im trying to teach maggie that nature doesnt e...,ko vtakpi vq vgcej ociikg vjcv pcvwtg fqgupv g...
4,its like an ox only it has a hump and a dewlap...,kvu nkmg cp qz qpna kv jcu c jwor cpf c fgyncr...


In [7]:
def convert_to_torch(text):
    output = torch.zeros((len(text), MAX_LEN), dtype=int)
    for i in range(len(text)):
        for j, w in enumerate(text[i]):
            if j >= MAX_LEN:
                break
            output[i, j] = CHAR_TO_INDEX.get(w, CHAR_TO_INDEX['none'])

    return output

In [8]:
train, test = train_test_split(df, test_size=0.2)
train_text = [[c for c in ph] for ph in train['text_out'] if type(ph) is str]
train_label= [[c for c in ph] for ph in train['text_in'] if type(ph) is str]
test_text   = [[c for c in ph] for ph in test['text_out'] if type(ph) is str]
test_label  = [[c for c in ph] for ph in test['text_in'] if type(ph) is str]

X_train= convert_to_torch(train_text)
Y_train= convert_to_torch(train_label)
X_test = convert_to_torch(test_text)
Y_test = convert_to_torch(test_label)

In [9]:
class RNN_Network(torch.nn.Module):
    def __init__(self):
        super(RNN_Network, self).__init__()
        self.embeddings = torch.nn.Embedding(len(ABC), 28)
        self.rnn = torch.nn.RNN(28, 256, batch_first=True)
        self.linear = torch.nn.Linear(256, 28)

    def forward(self, sentences, state=None):
        embds = self.embeddings(sentences)
        out, new_state = self.rnn(embds, state)
        result = self.linear(out)
        return result, new_state

In [18]:
model = RNN_Network().to(dev)
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
epochs = 40
loss_best = 10**10
model_best = None

In [19]:
for epoch in range(epochs):
    start = time.time()
    train_loss = 0.
    train_passed = 0
    test_loss = 0
    test_passed = 0

    for i in range(int(len(X_train) / 100)):
        X_batch = X_train[i * 100:(i + 1) * 100].to(dev)
        Y_batch = Y_train[i * 100:(i + 1) * 100].flatten().to(dev)
        model.train()
        optimizer.zero_grad()
        answers, _ = model.forward(X_batch)
        answers = answers.view(-1, len(ABC))
        loss = criterion(answers, Y_batch).to(dev)

        if loss < loss_best:
            model_best = copy.copy(model).to(dev)
            loss_best = loss.to(dev)

        train_loss += loss.item()
        loss.backward()
        optimizer.step()
        train_passed += 1

    with torch.no_grad():
        model.eval()
        answers, _ = model.forward(X_test.to(dev))
        answers = answers.view(-1, len(ABC))
        loss = criterion(answers, Y_test.flatten().to(dev))
        test_loss += loss.item()
        test_passed += 1

    if epoch%1 == 0:
        print(f"Epoch {epoch}. Time: {time.time() - start:.3f}, Train loss: {train_loss / train_passed:.3f}, Test loss: {test_loss / test_passed:.6f}")


Epoch 0. Time: 0.432, Train loss: 2.379, Test loss: 1.818156
Epoch 1. Time: 0.306, Train loss: 1.563, Test loss: 1.327599
Epoch 2. Time: 0.307, Train loss: 1.118, Test loss: 0.921128
Epoch 3. Time: 0.304, Train loss: 0.760, Test loss: 0.615860
Epoch 4. Time: 0.304, Train loss: 0.505, Test loss: 0.408253
Epoch 5. Time: 0.306, Train loss: 0.336, Test loss: 0.273480
Epoch 6. Time: 0.315, Train loss: 0.227, Test loss: 0.188010
Epoch 7. Time: 0.303, Train loss: 0.159, Test loss: 0.134049
Epoch 8. Time: 0.304, Train loss: 0.115, Test loss: 0.099491
Epoch 9. Time: 0.303, Train loss: 0.087, Test loss: 0.076642
Epoch 10. Time: 0.314, Train loss: 0.068, Test loss: 0.060909
Epoch 11. Time: 0.340, Train loss: 0.055, Test loss: 0.049623
Epoch 12. Time: 0.354, Train loss: 0.045, Test loss: 0.041231
Epoch 13. Time: 0.346, Train loss: 0.037, Test loss: 0.034802
Epoch 14. Time: 0.340, Train loss: 0.032, Test loss: 0.029755
Epoch 15. Time: 0.338, Train loss: 0.027, Test loss: 0.025711
Epoch 16. Time: 0.

In [20]:
loss_best

tensor(0.0020, device='cuda:0', grad_fn=<NllLossBackward0>)

In [21]:
pred = convert_to_torch([[c for c in s] for s in df['text_out'] if type(s) is str])

pred

tensor([[13, 10, 17,  ...,  0,  0,  0],
        [ 3, 14, 14,  ...,  0,  0,  0],
        [27, 14, 14,  ...,  0,  0,  0],
        ...,
        [20, 11, 22,  ...,  0,  0,  0],
        [13, 10,  4,  ...,  9,  3, 14],
        [10, 20,  9,  ...,  0,  0,  0]])

In [34]:
df_short = df.loc[:50]
df_short.shape[0]

51

In [35]:
texts = []
for line in range(df_short.shape[0]):
    str = ''

    for i in model_best(pred.to(dev))[0][line].argmax(dim=1).detach():
        str += INDEX_TO_CHAR[i.item()]

    texts.append(str)

df_short['predict'] = texts
df_short

<ipython-input-35-06bcc09a47b3>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_short['predict'] = texts


,text_in,text_out,predict
0,maggie look whats that,ociikg nqqm yjcvu vjcv,maggie look whats thatnonenonenonenonenonenone...
1,lee mur lee mur,ngg owt ngg owt,lee mur lee murnonenonenonenonenonenonenonenon...
2,zee boo zee boo,bgg dqq bgg dqq,zee boo zee boononenonenonenonenonenonenonenon...
3,im trying to teach maggie that nature doesnt e...,ko vtakpi vq vgcej ociikg vjcv pcvwtg fqgupv g...,im trying to teach maggie that nature doesnt e...
4,its like an ox only it has a hump and a dewlap...,kvu nkmg cp qz qpna kv jcu c jwor cpf c fgyncr...,its like an ox only it has a hump and a dewlap...
5,you know his blood type how romantic,aqw mpqy jku dnqqf varg jqy tqocpvke,you know his blood type how romanticnonenoneno...
6,oh yeah whats my shoe size,qj agcj yjcvu oa ujqg ukbg,oh yeah whats my shoe sizenonenonenonenonenone...
7,ring,tkpi,ringnonenonenonenonenonenonenonenonenonenoneno...
8,yes dad,agu fcf,yes dadnonenonenonenonenonenonenonenonenonenon...
9,ooh look maggie what is that do dec ah edron d...,qqj nqqm ociikg yjcv ku vjcv fq fge cj gftqp f...,ooh look maggie what is that do dec ah edron d...


### Задание 2

In [44]:
df

,text_in,text_out
0,maggie look whats that,ociikg nqqm yjcvu vjcv
1,lee mur lee mur,ngg owt ngg owt
2,zee boo zee boo,bgg dqq bgg dqq
3,im trying to teach maggie that nature doesnt e...,ko vtakpi vq vgcej ociikg vjcv pcvwtg fqgupv g...
4,its like an ox only it has a hump and a dewlap...,kvu nkmg cp qz qpna kv jcu c jwor cpf c fgyncr...
...,...,...
11634,too bad we didnt come dressed as popular carto...,vqq dcf yg fkfpv eqog ftguugf cu rqrwnct ectvq...
11635,yeah mom guess what for a dollar a man sold me...,agcj oqo iwguu yjcv hqt c fqnnct c ocp uqnf og...
11636,hows it going bart,jqyu kv iqkpi dctv
11637,maybe you need to play on their sympathies mor...,ocadg aqw pggf vq rnca qp vjgkt uaorcvjkgu oqt...


In [56]:
X = convert_to_torch([[c for c in s] for s in df['text_in'].tolist()])
X

tensor([[23,  4, 14,  ...,  0,  0,  0],
        [16,  1,  1,  ...,  0,  0,  0],
        [26,  1,  1,  ...,  0,  0,  0],
        ...,
        [12, 13,  8,  ...,  0,  0,  0],
        [23,  4, 22,  ...,  9, 16,  1],
        [ 4, 12,  9,  ...,  0,  0,  0]])

In [73]:
class Network(torch.nn.Module):
    def __init__(self):
        super(Network, self).__init__()
        self.embed = torch.nn.Embedding(len(CHAR_TO_INDEX), 28)
        self.rnn = torch.nn.RNN(28, 812, batch_first=True)
        self.linear = torch.nn.Linear(812, len(INDEX_TO_CHAR))

    def forward(self, sentences, state=None):
        embed = self.embed(sentences)
        o, s = self.rnn(embed)
        out = self.linear(o)
        return out


def generate_sentence(txt_list):
    sentence =  [c for c in txt_list]
    x = torch.zeros((1, len(sentence)), dtype=int).to(dev)

    for j,w in enumerate(sentence):
        if j >= MAX_S_LEN:
            break
        x[0, j] = CHAR_TO_INDEX.get(w, CHAR_TO_INDEX['none'])

    for i in range(MAX_LEN):
        o = model(x)
        w = torch.argmax(o[-1, -1, :], keepdim=True)
        x = torch.cat([x, w.unsqueeze(0)], axis=1)
        ww = INDEX_TO_CHAR[w.item()]
        if ww == 'none':
            break

        sentence.append(ww)

    return ''.join(sentence)


model = Network().to(dev)
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=.0001)
MAX_S_LEN = 100
phrase = 'bart i do so much for you'

In [61]:
len(X)

10891

In [74]:
for ep in range(200):
    start = time.time()
    train_loss = 0.
    train_passed = 0

    for i in range(int(len(X) / 100)):
        batch = X[i * 100:(i + 1) * 100]
        X_batch = batch[:, :-1].to(dev)
        Y_batch = batch[:, 1:].flatten().to(dev)

        optimizer.zero_grad()
        answers = model(X_batch)
        answers = answers.view(-1, len(INDEX_TO_CHAR))
        loss = criterion(answers, Y_batch).to(dev)
        train_loss += loss.item()

        loss.backward()
        optimizer.step()
        train_passed += 1

    if ep%10 == 0:
        print("\nEpoch {}. Time: {:.3f}, Train loss: {:.3f}".format(ep, time.time() - start, train_loss / train_passed))
        s = generate_sentence(phrase)
        print(s)
    else:
        print(f"\rEpoch {ep}, loss: {train_loss / train_passed:.3f}", end='')


Epoch 0. Time: 1.480, Train loss: 2.115
bart i do so much for you  oo  oo  oo  oo  oo  oo  oo  oo  oo  oo  oo  oo  
Epoch 9, loss: 1.428
Epoch 10. Time: 1.414, Train loss: 1.407
bart i do so much for you dont wall the soull the soull the soull the soull
Epoch 19, loss: 1.253
Epoch 20. Time: 1.410, Train loss: 1.239
bart i do so much for you sound the stopes the stopes the stopes the stopes
Epoch 29, loss: 1.136
Epoch 30. Time: 1.388, Train loss: 1.127
bart i do so much for your bart what i say what are you dont want to be a s
Epoch 39, loss: 1.056
Epoch 40. Time: 1.379, Train loss: 1.049
bart i do so much for your propest thing the store of the sture the store o
Epoch 49, loss: 0.997
Epoch 50. Time: 1.396, Train loss: 0.991
bart i do so much for your protest thing is the bart what i was stop it was
Epoch 59, loss: 0.949
Epoch 60. Time: 1.410, Train loss: 0.945
bart i do so much for your protest thing is the bart when i say the stare t
Epoch 69, loss: 0.909
Epoch 70. Time: 1.417, Train